In [8]:
import pandas as pd
import numpy as np

In [9]:
residences = pd.DataFrame(columns=['id','address', 'zipcode', 'latitude', 'longitude'])
survivors = pd.DataFrame(columns=['id', 'name', 'contact_1', 'contact_2', 'contact_3'])
contact_points = pd.DataFrame(columns = ['residence_id', 'survivor_id', 'date', 'water_lvl', 'damage_lvl', 'notes', 'intaker'])

In [10]:
def find_or_add_residence(address):
    global residences
    address = address.upper().strip()
    if residences.shape[0]==0:
        residences = residences.append({'id':0,'address':address, 'zipcode':np.nan, 'latitude':np.nan, 'longitude':np.nan}, ignore_index=True)
        res_id = 0
    else:
        try:
            res_id = residences.loc[residences['address'] == address,'id'].values[0]
        except:
            res_id = residences.shape[0]
            residences.loc[res_id,'id'] = res_id
            residences.loc[res_id,'address'] = address
    return res_id

In [11]:
def find_or_add_survivor(name, **kwargs):
    global survivors
    if name == None:
        return None
    else:
        name = name.upper().strip()
        try:
            surv_id = people[people['name']==name].index
        except:
            people.iloc[people.shape[0],0] = name
            surv_id = people[people['name'] == address].index
        return surv_id
        

In [12]:
def create_contact_point(**kwargs):
    global contact_points
    contact_points = contact_points.append(kwargs, ignore_index=True)
    

In [13]:
nash_codes = pd.read_csv('../data/CodesDamageAssessment_4_15_21.csv', parse_dates=['CreationDate'], 
                         dtype={'Address':'object','Extent of Damage':'object','Type of Structure':'object',
                                'Water Level':'float64','Note':'object','Creator':'object'})
nash_codes = nash_codes.dropna(subset=['Address'])

mask = (nash_codes['Type of Structure']=='Residential') & (nash_codes['Extent of Damage'] != 'None')
for ind, row in nash_codes[mask].iterrows():
    res_id = find_or_add_residence(row['Address'])
    surv_id = find_or_add_survivor(None)
    create_contact_point(residence_id = res_id, survivor_id = np.nan, 
                         date = row['CreationDate'], water_lvl = row['Water Level'], 
                         damage_lvl = row['Extent of Damage'], notes=row['Notes'], 
                         intaker='Nashville Codes: ' + row['Creator'])

In [20]:
oem = pd.read_csv('../data/OEM_4_12_21.csv', 
                         dtype={'PropHouse':'object','PropZip':'object','PropStreet':'object',
                                'Extent of Damage':'object','Water Level':'object','Notes':'object',
                                'Damage 2':'object'})

oem = oem.fillna('')

mask = oem['Extent of Damage'] != 'None'
for ind, row in oem[mask].iterrows():
    address = row['PropHouse'] + ' ' + row['PropStreet']
    res_id = find_or_add_residence(address)
    surv_id = None
    create_contact_point(residence_id = res_id, survivor_id = surv_id, date = pd.to_datetime('2021-04-12'), 
                         water_lvl = row['Water Level'], damage_lvl = row['Extent of Damage'], 
                         notes=row['Notes'] + row['Damage 2'], intaker='OEM')

In [21]:
contact_points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639 entries, 0 to 638
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   residence_id  639 non-null    object        
 1   survivor_id   0 non-null      object        
 2   date          639 non-null    datetime64[ns]
 3   water_lvl     502 non-null    object        
 4   damage_lvl    634 non-null    object        
 5   notes         296 non-null    object        
 6   intaker       639 non-null    object        
dtypes: datetime64[ns](1), object(6)
memory usage: 35.1+ KB
